Obtención de los datos

In [21]:
import pandas as pd

# Cargar datos
path='C:/4_F5/011_proyecto_ml/'


df = pd.read_csv(path + '/airline_passenger_satisfaction.csv')

#Se elimina la primera columna que era el subíndice del csv
df = df.iloc[:, 1:] 
df=df.drop(columns='id') #no sé si al final quito el id porque me estorba

In [22]:
#Hay que crear una parte que sea 'test'
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2, random_state=22 )

# test_size=0.2, este parámetro indica que deseas que el 20% de los datos se utilicen como conjunto de prueba y el 80% como conjunto de entrenamiento. 
# random_state=22 Este parámetro establece una semilla para el generador de números aleatorios que se usa para realizar la división. Esto asegura que cada vez que ejecutes el código, obtendrás la misma división de los datos en conjuntos de entrenamiento y prueba. Usar un valor de random_state fijo es una buena práctica cuando quieres que tus resultados sean reproducibles.

train.to_csv(path+'df_train.csv', index=False)
test.to_csv(path+'df_test.csv', index=False)
#Este test no lo tocamos hasta el final

3. Exploración y visualización de Datos

In [23]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103904 entries, 0 to 103903
Data columns (total 23 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   Gender                             103904 non-null  object 
 1   Customer Type                      103904 non-null  object 
 2   Age                                103904 non-null  int64  
 3   Type of Travel                     103904 non-null  object 
 4   Class                              103904 non-null  object 
 5   Flight Distance                    103904 non-null  int64  
 6   Inflight wifi service              103904 non-null  int64  
 7   Departure/Arrival time convenient  103904 non-null  int64  
 8   Ease of Online booking             103904 non-null  int64  
 9   Gate location                      103904 non-null  int64  
 10  Food and drink                     103904 non-null  int64  
 11  Online boarding                    1039

In [24]:
print(f'\n\nEl DF tiene {df.shape[0]:,} filas')
print(f'El DF tiene {df.shape[1]} columnas')



El DF tiene 103,904 filas
El DF tiene 23 columnas


In [25]:
# Comprobar cuántos valores missing existen en el DataFrame por variable

nulos={'conteo_nulos':df.isnull().sum(),'proporcion%':round(df.isnull().sum()/df.shape[0]*100, 3)}

df_nulos=pd.DataFrame(data=nulos)
print(df_nulos)

                                   conteo_nulos  proporcion%
Gender                                        0        0.000
Customer Type                                 0        0.000
Age                                           0        0.000
Type of Travel                                0        0.000
Class                                         0        0.000
Flight Distance                               0        0.000
Inflight wifi service                         0        0.000
Departure/Arrival time convenient             0        0.000
Ease of Online booking                        0        0.000
Gate location                                 0        0.000
Food and drink                                0        0.000
Online boarding                               0        0.000
Seat comfort                                  0        0.000
Inflight entertainment                        0        0.000
On-board service                              0        0.000
Leg room service        

In [26]:
# Conteo de los valores por cada columna ----> Hacer bucle para ver si hay columnas que estén desbalanceadas.
# Para ver qué variables hay que escalar

df['Customer Type'].value_counts()

Customer Type
Loyal Customer       84923
disloyal Customer    18981
Name: count, dtype: int64

Antes de convertir las variables categóricas, hay que tratar los 'missings'.

Que según hemos concluido, tantos los valores '0' como los 'espacios en blanco' son 'missing' y los vamos a eliminar.

In [27]:
# Primero elimino los 'missings'
df = df.dropna(how='any')

print(f'\n\nEl DF tiene {df.shape[0]:,} filas')


# Compruebo que se ha eliminado
nulos={'conteo_nulos':df.isnull().sum(),'proporcion%':round(df.isnull().sum()/df.shape[0]*100, 3)}
df_nulos=pd.DataFrame(data=nulos)
print(df_nulos)



El DF tiene 103,594 filas
                                   conteo_nulos  proporcion%
Gender                                        0          0.0
Customer Type                                 0          0.0
Age                                           0          0.0
Type of Travel                                0          0.0
Class                                         0          0.0
Flight Distance                               0          0.0
Inflight wifi service                         0          0.0
Departure/Arrival time convenient             0          0.0
Ease of Online booking                        0          0.0
Gate location                                 0          0.0
Food and drink                                0          0.0
Online boarding                               0          0.0
Seat comfort                                  0          0.0
Inflight entertainment                        0          0.0
On-board service                              0          

In [28]:
#Ahora elimino los '0' pero de las columnas categóricas ordinales



In [29]:
import pandas as pd
from pandas.api.types import is_categorical_dtype

# Función para detectar variables categóricas ordinales
def detectar_variables_ordinales(df):
    ordinal_columns = []

    for column in df.columns:
        # verificar si la columna es numérica
        if pd.api.types.is_numeric_dtype(df[column]):
            # obtener valores únicos
            unique_values = df[column].unique()
            # verificar si todos los valores están en el rango [1,5]
            if set(unique_values).issubset({0, 1, 2, 3, 4, 5}):
                ordinal_columns.append(column)

    
    return ordinal_columns




variables_ordinales = detectar_variables_ordinales(df)
print("Variables categóricas ordinales:", variables_ordinales)
rows_with_zero = (df[variables_ordinales] == 0).any(axis=1).sum()
print(rows_with_zero)

print(len(variables_ordinales))
# Resultado:
# Variables categóricas ordinales: ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']
# De esas variables hay que quitar los '0' porque hemos decidido que son missing. Antes de hacer el one-hot-encoding


Variables categóricas ordinales: ['Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']
8179
14


In [30]:
y = df['satisfaction']

In [31]:
# Variables que son categóricas binarias y categóricas no ordinales
import pandas as pd

def detectar_variables_categoricas(df):
    categorias_columns = []

    for column in df.columns:
        # Verificar si la columna no es numérica
        if not pd.api.types.is_numeric_dtype(df[column]):
            # Obtener el número de categorías únicas
            unique_values = df[column].nunique()
            # Se puede agregar un límite arbitrario si se desea,
            # por ejemplo, menos de 10 categorías únicas
            if unique_values < 10:
                categorias_columns.append(column)

    return categorias_columns

# Supongamos que tienes un DataFrame df
variables_categoricas = detectar_variables_categoricas(df)
print(variables_categoricas)

['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction']


In [ ]:
#Filtro para quedarme con las numéricas, las que voy a escalar

# Primero detecto las columnas que tienen solo valores de 0 y 1
""" binary_cols = x_encoded.columns[x_encoded.nunique() == 2]

# Luego detecto las columnas ordinales (2 a 5)
ordinal_cols = x_encoded.columns[(x_encoded.nunique() > 1) & (x_encoded.nunique() <= 5)] """

# Ambas categóricas las junto 
categorical_cols = list(variables_categoricas) + list(variables_ordinales)

print(categorical_cols)
print(len(categorical_cols))

Separo las variables categóricas de las variables numéricas

In [43]:
data_cat=df[categorical_cols]
data_num=df.drop(columns=categorical_cols)

print(data_cat)

        Gender      Customer Type   Type of Travel     Class  \
0         Male     Loyal Customer  Personal Travel  Eco Plus   
1         Male  disloyal Customer  Business travel  Business   
2       Female     Loyal Customer  Business travel  Business   
3       Female     Loyal Customer  Business travel  Business   
4         Male     Loyal Customer  Business travel  Business   
...        ...                ...              ...       ...   
103899  Female  disloyal Customer  Business travel       Eco   
103900    Male     Loyal Customer  Business travel  Business   
103901    Male  disloyal Customer  Business travel  Business   
103902  Female  disloyal Customer  Business travel       Eco   
103903    Male     Loyal Customer  Business travel  Business   

                   satisfaction  Inflight wifi service  \
0       neutral or dissatisfied                      3   
1       neutral or dissatisfied                      3   
2                     satisfied                      2   

In [33]:
""" count_zeros = (df == 0).sum().sum()
print(count_zeros) """

# Contar los valores iguales a 0 en las columnas ordinales
conteo_ceros = (df[variables_ordinales] == 0).sum()   ## conteo_ceros = df[variables_ordinales].apply(lambda x: (x == 0).sum())

# Sumar el conteo para obtener el total de ceros en todas las columnas
total_ceros = conteo_ceros.sum()

# Mostrar
print("\nConteo de '0's por columna:\n\n", conteo_ceros)
print(f"\nTotal de '0's en todas las columnas: {total_ceros:,}")



Conteo de '0's por columna:

 Inflight wifi service                3096
Departure/Arrival time convenient    5290
Ease of Online booking               4473
Gate location                           1
Food and drink                        105
Online boarding                      2420
Seat comfort                            1
Inflight entertainment                 14
On-board service                        3
Leg room service                      470
Baggage handling                        0
Checkin service                         1
Inflight service                        3
Cleanliness                            12
dtype: int64

Total de '0's en todas las columnas: 15,889


In [34]:
#df_sin_ceros = df[(df[variables_ordinales] != 0).all(axis=1)]#.reset_index()

df_sin_ceros = df

In [35]:
""" for col in variables_ordinales:
    df_sin_ceros = df[df[col] != 0] #df_sin_ceros = df[(df != 0).all(axis=1)]
    
     """
    
    
#Vuelto a contar los '0's en el dataframe de las variables ordinales para comprobar que se han eliminado
conteo_ceros = (df_sin_ceros[variables_ordinales] == 0).sum()   ## conteo_ceros = df[variables_ordinales].apply(lambda x: (x == 0).sum())

#¿Porque predice mal? o porque está sesgado (es inherente a la muestra tomada)
# Aleatorio no es lo mismo que representativa

# Sumar el conteo para obtener el total de ceros en todas las columnas
total_ceros = conteo_ceros.sum()

# Mostrar
print("\nConteo de ceros por columna:\n\n", conteo_ceros)
print("\nTotal de ceros en todas las columnas:", total_ceros)

print(f'Quedan {df_sin_ceros.shape[0]:,} filas')
# Contar las filas que contienen algún 0 en las columnas especificadas
rows_with_zero = (df[variables_ordinales] == 0).any(axis=1).sum()

print(f"Número de filas que tenían algún '0' en las columnas ordinales: {rows_with_zero}")


Conteo de ceros por columna:

 Inflight wifi service                3096
Departure/Arrival time convenient    5290
Ease of Online booking               4473
Gate location                           1
Food and drink                        105
Online boarding                      2420
Seat comfort                            1
Inflight entertainment                 14
On-board service                        3
Leg room service                      470
Baggage handling                        0
Checkin service                         1
Inflight service                        3
Cleanliness                            12
dtype: int64

Total de ceros en todas las columnas: 15889
Quedan 103,594 filas
Número de filas que tenían algún '0' en las columnas ordinales: 8179


Antes de hacer la matriz de correlación, hay que convertir las columnas que son 'object' a binarias.

(pero cuidado, PRIMERO hay que separar la variable objetivo, que es una columna del dataframe)

**One-Hot Encoding es generalmente preferido cuando no hay un orden inherente en las categorías. De esta manera, me aseguro de que el modelo no asuma relaciones incorrectas entre las categorías.

**Label Encoding podría usarse si hay un orden claro entre las categorías

In [36]:
import category_encoders as ce
from sklearn.preprocessing import OrdinalEncoder
# Separar la variable objetivo antes de la codificación, la llamo 'y'.
# Si no la separo todavía, la va a codificar, lo cual no sé si conviene por el momento.
#y = df_sin_ceros['satisfaction']

#-- Codificar solo las variables categóricas--

# Variables categóricas
encoder_cat = ce.OneHotEncoder(use_cat_names=True)
x_encoded = encoder_cat.fit_transform(df_sin_ceros.drop(columns=['satisfaction']))
print(x_encoded)


# Variables categóricas ordinales
# Crear OrdinalEncoder
encoder_ord = OrdinalEncoder()
# Aplicar OrdinalEncoder a múltiples variables categóricas ordinales.
# Machaco la variable x_encoded que la recojo de la codificación anterior.
x_encoded  = pd.DataFrame(encoder_ord.fit_transform(x_encoded), columns=x_encoded.columns)



# Concatenar la columna 'satisfaccion' de nuevo al DataFrame codificado
df_encoded = pd.concat([x_encoded, y], axis=1)

""" # Dividir los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x_encoded, y, test_size=0.2, random_state=42)

# Guardar datos en ficheros
path='C:/4_F5/011_proyecto_ml/'
x_train.to_csv(path + '/x_train.csv')
x_test.to_csv(path + '/x_test.csv')
y_train.to_csv(path + '/y_train.csv')
y_test.to_csv(path + '/y_test.csv')
 """



        Gender_Male  Gender_Female  Customer Type_Loyal Customer  \
0                 1              0                             1   
1                 1              0                             0   
2                 0              1                             1   
3                 0              1                             1   
4                 1              0                             1   
...             ...            ...                           ...   
103899            0              1                             0   
103900            1              0                             1   
103901            1              0                             0   
103902            0              1                             0   
103903            1              0                             1   

        Customer Type_disloyal Customer  Age  Type of Travel_Personal Travel  \
0                                     0   13                               1   
1                      

" # Dividir los datos en conjuntos de entrenamiento y prueba\nx_train, x_test, y_train, y_test = train_test_split(x_encoded, y, test_size=0.2, random_state=42)\n\n# Guardar datos en ficheros\npath='C:/4_F5/011_proyecto_ml/'\nx_train.to_csv(path + '/x_train.csv')\nx_test.to_csv(path + '/x_test.csv')\ny_train.to_csv(path + '/y_train.csv')\ny_test.to_csv(path + '/y_test.csv')\n "


Cuando tienes un DataFrame con columnas categóricas y categóricas ordinales, es importante elegir el método de codificación adecuado que preserve la naturaleza ordinal de la columna.

In [37]:
import janitor  # Esto importa pyjanitor y extiende pandas

Voy a separar las variables, por un lado según sean categóricas(binarias, ordinales..) y por otro las numéricas

· Categóricas

· Numéricas

Porque a veces un modelo va a trabajar mejor sólo con las categóricas o sólo con las numéricas.
Además, es conveniente escalar las variables munéricas (ya sean por cambios de escala o por magnitud, o incluso 'normalizarlas')

In [38]:
""" #Filtro para quedarme con las numéricas, las que voy a escalar

# Primero detecto las columnas que tienen solo valores de 0 y 1
binary_cols = x_encoded.columns[x_encoded.nunique() == 2]

# Luego detecto las columnas ordinales (2 a 5)
ordinal_cols = x_encoded.columns[(x_encoded.nunique() > 1) & (x_encoded.nunique() <= 5)]

# Ambas categóricas las junto 
categorical_cols = list(variables_categoricas) + list(variables_ordinales)

print(categorical_cols)
print(len(categorical_cols)) """

['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction', 'Inflight wifi service', 'Departure/Arrival time convenient', 'Ease of Online booking', 'Gate location', 'Food and drink', 'Online boarding', 'Seat comfort', 'Inflight entertainment', 'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Inflight service', 'Cleanliness']
19


¿Por qué este orden es importante?

Consistencia: Al separar primero X e y y luego hacer el split, garantizas que las filas correspondientes de X e y coincidan exactamente en los conjuntos de entrenamiento y prueba.

Evitar filtraciones de datos: Si haces el split antes de separar X e y, podrías accidentalmente introducir fugas de datos o inconsistencias, lo que podría afectar la capacidad del modelo para generalizar.

* Primero separas la 'y' de las variables predictoras 'x'.
* Luego El split de tanto 'y' como 'x' con su correspondiente 'train' y 'test'
* Finalmente voy a escalar las numéricas: Después de realizar el split, ajustas el escalador (Scaler) solo con el conjunto de entrenamiento (X_train). Esto asegura que las estadísticas de escalado (media, desviación estándar, etc.) no estén influenciadas por los datos de prueba.
* Aplicar la transformación de escalado al conjunto de prueba:Una vez que el escalador se ha ajustado al conjunto de entrenamiento, usas ese mismo escalador para transformar el conjunto de prueba (X_test). Esto asegura que ambos conjuntos sean escalados de manera consistente


In [39]:
#Primero separas la 'y' de las variables predictoras 'x'
#Luego El split de tanto 'y' como 'x' con su correspondiente 'train' y 'test'
#Finalmente voy a escalar las numéricas: Después de realizar el split, ajustas el escalador (Scaler) solo con el conjunto de entrenamiento (X_train). Esto asegura que las estadísticas de escalado (media, desviación estándar, etc.) no estén influenciadas por los datos de prueba.
#Aplicar la transformación de escalado al conjunto de prueba:Una vez que el escalador se ha ajustado al conjunto de entrenamiento, usas ese mismo escalador para transformar el conjunto de prueba (X_test). Esto asegura que ambos conjuntos sean escalados de manera consistente
from sklearn.preprocessing import StandardScaler


print(f'Las variables categóricas son {len(categorical_cols)}:\n\n' + '\n'.join(categorical_cols) + '\n')


# Separa el dataframe en partes de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(x_encoded, y, test_size=0.3, random_state=42)

""" categorical_cols.remove('satisfaction')
print(categorical_cols) """

print(x_train.columns)
print(len(x_train.columns))



Las variables categóricas son 19:

Gender
Customer Type
Type of Travel
Class
satisfaction
Inflight wifi service
Departure/Arrival time convenient
Ease of Online booking
Gate location
Food and drink
Online boarding
Seat comfort
Inflight entertainment
On-board service
Leg room service
Baggage handling
Checkin service
Inflight service
Cleanliness

Index(['Gender_Male', 'Gender_Female', 'Customer Type_Loyal Customer',
       'Customer Type_disloyal Customer', 'Age',
       'Type of Travel_Personal Travel', 'Type of Travel_Business travel',
       'Class_Eco Plus', 'Class_Business', 'Class_Eco', 'Flight Distance',
       'Inflight wifi service', 'Departure/Arrival time convenient',
       'Ease of Online booking', 'Gate location', 'Food and drink',
       'Online boarding', 'Seat comfort', 'Inflight entertainment',
       'On-board service', 'Leg room service', 'Baggage handling',
       'Checkin service', 'Inflight service', 'Cleanliness',
       'Departure Delay in Minutes', 'Arrival Dela

In [40]:
# Filtro o identifico cuáles son las columnas categóricas de 'x' y quedarme con las numéricas, porque son esas las que voy a escalar
x_num_train = x_train.drop(columns=categorical_cols).columns #Aquí Sólo se están guardando el nombre de las columnas para rastrearlas
pd.DataFrame(x_train[x_num_train]).to_csv(path+'/x_num_train_no_scaled.csv', index=False)

x_num_test = x_test.drop(columns=categorical_cols).columns #Sólo cojo el nombre de las columnas para rastrearlas
pd.DataFrame(x_test[x_num_test]).to_csv(path+'/x_num_test_no_scaled.csv', index=False)


print(f'Las variables numéricas son {x_num_train.shape[0]}:\n\n' + '\n'.join(x_num_train) + '\n')

# Crea un objeto StandardScaler y ajusta los parámetros de escalado con la parte de entrenamiento
scaler = StandardScaler()
x_train_num_scaled = scaler.fit_transform(x_train[x_num_train])

# Aplica los parámetros de escalado aprendidos a la parte de prueba
x_test_num_scaled = scaler.transform(x_test[x_num_test]) #devuelve un array de arrays

KeyError: "['Gender', 'Customer Type', 'Type of Travel', 'Class', 'satisfaction'] not found in axis"

La razón es que, en general, deseas mantener la separación entre los datos de entrenamiento y los datos de prueba, incluso después de la escalado. De esta manera, puedes asegurarte de que el modelo de machine learning se entrena y se evalúa correctamente.

In [ ]:

# En 'train' cojo las columnas numéricas sin escalar y las reemplazo con las columnas numéricas escaladas. 
# Por lo que ya me queda todo 'x_train' junto y escalado
x_train[x_num_train] = x_train_num_scaled 
print(x_train)

x_train.to_csv(path +   '/x_train_scaled.csv')

# En 'test' cojo las columnas numéricas sin escalar y las reemplazo con las columnas numéricas escaladas.
# Por lo que ya me queda todo 'x_test' junto y escalado
x_test[x_num_test] = x_test_num_scaled
x_test.to_csv(path + '/x_test_scaled.csv')

print(x_test)

In [ ]:
# Guardar datos en ficheros
""" 
path='C:/4_F5/011_proyecto_ml/'
pd.DataFrame(x_train_scaled).to_csv('x_train_scaled.csv', index=False)
pd.DataFrame(x_test_scaled).to_csv('x_test_scaled.csv', index=False)
y_train.to_csv(path + '/y_train.csv')
y_test.to_csv(path + '/y_test.csv')

 """

In [ ]:
import matplotlib.pyplot as plt

# Histograma de la variable escalada
plt.hist(x_test['Age'], bins=50)
plt.title('Distribución de la variable escalada')
plt.show()

# Histograma de la variable no escalada ---> usar de A´ngel


In [ ]:
print('Estadísticas descriptivas de la variable escalada:')
print(x_test['Age'].describe())

print('Estadísticas descriptivas de la variable escalada:')
#print(x_test_scaled[:, 0]).describe()) 

Primer modelo

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Crea un objeto RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entrena el modelo con el conjunto de entrenamiento
model.fit(x_train, y_train)

# Evalúa el modelo con el conjunto de prueba
y_pred = model.predict(x_test)
print("Accuracy:", accuracy_score(y_test, y_pred))

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

# ...

y_pred = model.predict(x_test)

# Calcula la accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


# Calcula la precisión (average='weighted' for multi-class problems)
precision = precision_score(y_test, y_pred, average='weighted')
print("Precision:", precision)

# Calcula el recall
recall = recall_score(y_test, y_pred, average='weighted',labels=['neutral or dissatisfied', 'satisfied'])
print("Recall:", recall)

""" # Calcula el F1-score
f1 = f1_score(y_test, y_pred)
print("F1-score:", f1)

# Calcula el ROC-AUC
roc_auc = roc_auc_score(y_test, y_pred)
print("ROC-AUC:", roc_auc)

# Calcula la matriz de confusión
conf_mat = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_mat) """

Multicolinealidad

Probando 

In [ ]:
""" 
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
import numpy as np


le = LabelEncoder()
y_train_le = le.fit_transform(y_train)

lin_reg = LinearRegression()
lin_reg.fit(x_train,y_train_le)


#print(x_train.head())
print(y_train.head())
print(lin_reg.predict(x_train.head()))

"""



""" 
El problema es que el modelo de regresión lineal (LinearRegression) no entiende que tu variable objetivo es binaria.

La regresión lineal es un algoritmo de aprendizaje automático que se utiliza para predecir valores continuos, no binarios. 
Cuando entrenas el modelo con tus datos, el algoritmo intenta encontrar la mejor línea que se ajuste a tus datos, pero no 
tiene en cuenta que la variable objetivo es binaria.

Por lo tanto, cuando haces print(lin_reg.predict(x_train.head())), el modelo devuelve valores decimales porque está tratando de predecir un valor continuo, no un valor binario.

Para solucionar este problema, debes utilizar un algoritmo de aprendizaje automático que sea adecuado para problemas de clasificación binaria, como LogisticRegression de scikit-learn. Este algoritmo entiende que la variable objetivo es binaria y devuelve probabilidades de pertenencia a cada clase, que puedes convertir en valores binarios (0 o 1) utilizando una función de umbral (threshold).

 """

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Codificaicón de la variable objetivo
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)


# Entrenamiento del modelo
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train_le)


# Predicción en el conjunto de prueba
y_test_le = le.transform(y_test)  # Codifica y_test para compararlo correctamente
y_pred = log_reg.predict(x_test)  # Predicción sobre el conjunto de prueba

# Evaluación del modelo
accuracy = accuracy_score(y_test_le, y_pred)
print("Accuracy:", accuracy)

# Ejemplo de predicciones sobre nuevas muestras (primeras 5 filas del conjunto de prueba)
print(x_test.head())
y_pred_example = log_reg.predict(x_test.head())
print(y_pred_example)



results_df = pd.DataFrame({
    'Real': y_test_le[:5],  # Primeras 5 filas reales
    'Predicción': y_pred_example # Primeras 5 predicciones
})

# Imprimir la tabla de resultados
print("\nValores reales vs. Predicciones:")
print(results_df)

En lugar de calcular la correlación entre las variables dummy, es más útil utilizar otras técnicas para seleccionar las variables más relevantes para el modelo. Algunas opciones son:

Análisis de la importancia de las características: utiliza algoritmos como Random Forest o Gradient Boosting para evaluar la importancia de cada variable en la predicción de Y.
Selección de características: utiliza técnicas como la selección recursiva de características (RFE) o la selección de características mediante la matriz de correlación de mutual information.
Análisis de la varianza: utiliza técnicas como la descomposición de la varianza para evaluar la contribución de cada variable a la varianza de Y.
Recuerda que la selección de variables es un paso importante en el proceso de modelado, y es importante utilizar técnicas adecuadas para identificar las variables más relevantes para el modelo.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

# Codificación de la variable objetivo
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_test_le = le.transform(y_test)  # Codifica y_test para compararlo correctamente

# Entrenamiento del modelo
log_reg = LogisticRegression()
log_reg.fit(x_train, y_train_le)

# Predicción en el conjunto de prueba
y_pred = log_reg.predict(x_test)  # Predicción sobre el conjunto de prueba

# Evaluación del modelo
accuracy = accuracy_score(y_test_le, y_pred)
print("Accuracy:", accuracy)




# Convertir las predicciones numéricas a etiquetas originales es por pura estética del 'satiesfied'
y_pred_example_labels = le.inverse_transform(y_pred_example)
print("\nPredicciones con etiquetas originales:")


y_test_example_labels = le.inverse_transform(y_test_le)


# Verificar si las predicciones son 'satisfied'
print("\n¿Las predicciones son 'satisfied'?")

# Mostrar los primeros 5 valores reales y predichos    
# Crear un DataFrame para mostrar los resultados
results_df = pd.DataFrame({
    'Real': y_test_example_labels[:5],  # Primeras 5 filas reales
    'Predicción': y_pred_example_labels[:5]  # Primeras 5 predicciones
})

# Imprimir la tabla de resultados
print("\nValores reales vs. Predicciones:")
print(results_df)

Voy a usar más métricas

In [ ]:
from sklearn.metrics import confusion_matrix
#
# Generar la matriz de confusión
cm = confusion_matrix(y_test_le, y_pred)

print("Matriz de Confusión:")
print(cm)

La matriz de confusión saca sus resultados a partir de lo parte de 'test', en general (pero se puede hacer una matriz de confusión de la parte de 'train' para comparar con la matriz de 'test' y así encontrar señales de 'overfitting'. Esto lo haré luego)

Para interpretar esta matriz, debemos entender qué representan cada una de las celdas:

La diagonal principal (de arriba a la izquierda a abajo a la derecha) representa las observaciones que el modelo clasificó correctamente. En este caso, hay 15005 observaciones que el modelo clasificó como negativas (0) y realmente eran negativas, y 10665 observaciones que el modelo clasificó como positivas (1) y realmente eran positivas.
La celda superior derecha (1455) representa las observaciones que el modelo clasificó como positivas (1) pero realmente eran negativas (0). Estas son llamadas "falsos positivos" (FP).
La celda inferior izquierda (1500) representa las observaciones que el modelo clasificó como negativas (0) pero realmente eran positivas (1). Estas son llamadas "falsos negativos" (FN).
A partir de esta matriz de confusión, podemos sacar varias conclusiones:

Precisión: La precisión del modelo se puede calcular como la suma de las observaciones clasificadas correctamente (diagonal principal) dividida entre el total de observaciones. En este caso, la precisión es (15005 + 10665) / (15005 + 1455 + 1500 + 10665) ≈ 0.92. Esto significa que el modelo es capaz de clasificar correctamente alrededor del 92% de las observaciones.

Sensibilidad: La sensibilidad se refiere a la capacidad del modelo para detectar observaciones positivas (1). Se puede calcular como la cantidad de verdaderos positivos (TP) dividida entre la suma de verdaderos positivos y falsos negativos (FN). En este caso, la sensibilidad es 10665 / (10665 + 1500) ≈ 0.877. Esto significa que el modelo es capaz de detectar alrededor del 87.7% de las observaciones positivas.

Especificidad: La especificidad se refiere a la capacidad del modelo para detectar observaciones negativas (0). Se puede calcular como la cantidad de verdaderos negativos (TN) dividida entre la suma de verdaderos negativos y falsos positivos (FP). En este caso, la especificidad es 15005 / (15005 + 1455) ≈ 0.912. Esto significa que el modelo es capaz de detectar alrededor del 91.2% de las observaciones negativas.
En resumen, la matriz de confusión indica que el modelo de regresión logística tiene una buena precisión y especificidad, pero una sensibilidad moderada. Esto sugiere que el modelo es capaz de clasificar correctamente la mayoría de las observaciones, pero puede tener dificultades para detectar algunas observaciones positivas.

Mirar qué es sobreajuste y cómo medirlo

El 'overfitting' es un problema que ocurre cuando un modelo de machine learning se ajusta demasiado bien a los datos de entrenamiento, capturando tanto las relaciones como el ruido o patrones irrelevantes en los datos.
Como resultado, aunque el modelo tenga un alto rendimiento en los datos de entrenamiento, su capacidad de generalización a nuevos datos o datos de prueba deficiente, lo que conduce a un bajo rendimiento de estos últimos.


En el caso del Modelo de Regresión Logística, algunas señales para evaluar Overfitting podría ser:
* Alta precisión en el conjunto de entrenamiento y baja precisión en el conjunto de prueba.
* Matriz de confusión desequilibrada, donde el modelo clasifica correctamente en el conjunto de entrenamiento, pero falla significativamente en el conjunto de prueba.
* Curva ROC/AUC, donde el AUC en los datos de prueba es significativamente más bajo que en los datos de entrenamiento.


Por tanto, se pueden realizar las siguientes comprobaciones para evaluarlo:


:

* 1. Compara el rendimiento entre los datos de entrenamiento y prueba: Revisa las métricas de precisión (accuracy), precisión (precision), sensibilidad (recall), y el AUC en ambos conjuntos de datos. Si el rendimiento es mucho mejor en el conjunto de entrenamiento que en el conjunto de prueba, es una señal clara de overfitting.

In [ ]:
# Predicción en el conjunto de entrenamiento
y_train_pred = log_reg.predict(x_train)
train_accuracy = accuracy_score(y_train_le, y_train_pred)

print("Precisión en el conjunto de entrenamiento:", train_accuracy)
print("Precisión en el conjunto de prueba:", accuracy) 

* 2. Curva de aprendizaje: 
Podemos graficar el rendimiento del modelo en el conjunto de entrenamiento y prueba en función del número de muestras. Si la precisión en el conjunto de prueba se estabiliza a un nivel bajo y en el conjunto de entrenamiento es muy alta, esto es una indicación de overfitting.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import learning_curve
from sklearn.linear_model import LogisticRegression

# Parámetros para la función learning_curve
train_sizes = np.linspace(0.1, 1.0, 10)  # Fracciones del conjunto de datos de entrenamiento que se usarán

# Generar curva de aprendizaje
train_sizes, train_scores, validation_scores = learning_curve(
    LogisticRegression(), 
    x_train, y_train_le, 
    train_sizes=train_sizes, 
    cv=5,  # Validación cruzada de 5 folds
    scoring='accuracy',  # Medimos la precisión del modelo
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Cálculo de la media y desviación estándar para la precisión de entrenamiento y validación
train_scores_mean = np.mean(train_scores, axis=1)
train_scores_std = np.std(train_scores, axis=1)
validation_scores_mean = np.mean(validation_scores, axis=1)
validation_scores_std = np.std(validation_scores, axis=1)

# Graficar la curva de aprendizaje
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, label='Precisión en entrenamiento', color='blue')
plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, color='blue', alpha=0.2)
plt.plot(train_sizes, validation_scores_mean, label='Precisión en validación', color='green')
plt.fill_between(train_sizes, validation_scores_mean - validation_scores_std, validation_scores_mean + validation_scores_std, color='green', alpha=0.2)

# Etiquetas y leyenda
plt.title('Curva de aprendizaje para Regresión Logística')
plt.xlabel('Tamaño del conjunto de entrenamiento')
plt.ylabel('Precisión')
plt.legend(loc='best')

# Mostrar gráfico
plt.grid()
plt.show()


Pruebo otro Modelo

4. Preparación de los Algoritmos de ML

5. Seleccionar un modelo y entrénalo

6. Ajustar el modelo (optimización con hypermaprámetros, ensamblado)

7. Presentar la solución.

8. Desplegar y monitorizar